Note that you're still focusing on the feature-engineering step of the data-preprocessing pipeline as shown below:

![Feature Engineering](assets/feature_engineering.png)



# Semantics

In the BoW approach, with all the information that you were able to pull out of the text, one thing that you didn't really use was semantics—the *meaning* of the words and sentences. The models that you built in the previous checkpoint may know that Jane Austen tends to use the word `lady` a lot in her writing, but they don't know what a lady is. There is nothing in your work on NLP so far that would allow a model to say whether `queen` or `car` is more similar to `lady`. 

In the absence of semantic information, models can get tripped up on things like synonyms (`milady` and `lady`). You could modify the spaCy dictionary to include `lady` as the lemma of `milady`, then use lemmas for all your analyses. But for this to be an effective approach, you would have to go through your entire corpus and identify all synonyms for all words by hand. This approach would also discard subtle differences in the connotations of words, concepts, ideas, or emotions associated with `lady` (which elicits thoughts of formal manners and England) and `milady` (which elicits thoughts of medieval ages and Renaissance Faires).

Language is complicated, and trying to explicitly model all the information encoded in a language is nearly impossibly complicated. Fortunately, there are some approaches and methods that you can use to get around this to some extent. In general, these methods work on a corpus of text and learn the rules of the language by identifying recurring patterns within the corpus. As the outcome, all of these methods produce a numerical representation of the words. 

In this checkpoint, you will learn about the *term frequency–inverse document frequency* (TF-IDF) method as a modification of the BoW approach of the previous checkpoint. You'll also learn about *latent semantic analysis*.


# BoW revisited: TF-IDF

The BoW approach rests upon counting the occurrences of the words in the documents (which in this case are the sentences). However, there is other information that you can exploit from the occurrences of the words, apart from their counts in the sentences. In the following, you'll go step by step as you learn about TF-IDF vectorization, which takes some clever steps when counting the words. 

Consider the following sentences:

1. "The best Monty Python sketch is the one about the dead parrot; I laughed so hard."
2. "I laugh when I think about Python's Ministry of Silly Walks sketch; it is funny, funny, funny, the best!"
3. "Chocolate is the best ice cream dessert topping, with a great taste."
4. "The Lumberjack Song is the funniest Monty Python bit; I can't think of it without laughing."
5. "I would rather put strawberries on my ice cream for dessert; they have the best taste."
6. "The taste of caramel is a fantastic accompaniment to tasty mint ice cream."

As a human being, it's easy to see that the sentences involve two topics: comedy and ice cream. One way to represent the sentences is in a *term-document matrix* that has a column for each sentence and a row for each word. Ignoring the stop words `the`, `is`, `and`, `a`, `of`, `I`, and `about`, discarding words that occur only once, and reducing words like `laughing` to their root form (`laugh`), the term-document matrix for these sentences would be as follows:

|           | 1 | 2 | 3 | 4 | 5 | 6 |
|-----------|---|---|---|---|---|---|
| Monty     | 1 | 0 | 0 | 1 | 0 | 0 |
| Python    | 1 | 1 | 0 | 1 | 0 | 0 |
| sketch    | 1 | 1 | 0 | 0 | 0 | 0 |
| laugh     | 1 | 1 | 0 | 1 | 0 | 0 |
| funny     | 0 | 3 | 0 | 1 | 0 | 0 |
| best      | 1 | 1 | 1 | 0 | 1 | 0 |
| ice cream | 0 | 0 | 1 | 0 | 1 | 1 |
| dessert   | 0 | 0 | 1 | 0 | 1 | 0 |
| taste     | 0 | 0 | 1 | 0 | 1 | 2 |



Note that the term *document* is used here to refer to the individual text chunks that you are working with. It can sometimes mean sentences, sometimes paragraphs, and sometimes whole text files. In your case, each sentence is a document. Also note that, contrary to how you've been operating, a term-document matrix has words as rows and documents as columns.

The comedy sentences use the following words: `Python` (3), `laugh` (3), `Monty` (2), `sketch` (2), `funny` (2), and `best` (2).
The ice cream sentences use the following words: `ice cream` (3), `dessert` (3), `taste` (3), and `best` (2).

The word `best` stands out here—it appears in more sentences than any other word (four out of the six sentences). It is used equally to describe Monty Python and ice cream. If you were to use this term-document matrix as is to teach a computer to parse sentences, `best` would end up as a significant identifier for both topics. And every time that you gave the model a new sentence to identify that included `best`, it would bring up both topics. Not very useful. To avoid this, you want to weight the matrix so that words that occur in many different sentences have lower weights than words that occur in fewer sentences. It is important to put a floor on this though—words that only occur once are totally useless for finding associations between sentences. 

Another word that stands out is `funny`, which appears more often in comedy sentences than any other word. This suggests that `funny` is a very important word for defining the *comedy* topic. 

## Quantifying documents: Collection and document frequencies

*Document frequency* (DF) counts how many sentences a word appears in. *Collection frequency* (CF) counts how often a word appears, total, over all sentences. Now, calculate the DF and CF for your sentence set:

|           |DF |CF| 
|-----------|---|---|
| Monty     | 2 | 2 | 
| Python    | 3 | 3 | 
| sketch    | 2 | 2 | 
| laugh     | 3 | 3 | 
| funny     | 2 | 4 | 
| best      | 4 | 4 | 
| ice cream | 3 | 3 | 
| dessert   | 2 | 2 | 
| taste     | 3 | 4 | 



## Penalizing indiscriminate words: Inverse document frequency

Now, weight the document frequency so that words that occur less often (like `sketch` and `dessert`) are more influential than words that occur a lot (like `best`). Below, you will calculate the ratio of total documents (*N*) divided by *DF*, then take the log (base 2) of the ratio, to get your *inverse document frequency* (IDF) number for each term (t):

$$idf_t=log \dfrac N{df_t}$$


|           |df |cf| idf |
|-----------|---|---|
| Monty     | 2 | 2 | 1.585 |
| Python    | 3 | 3 | 1 |
| sketch    | 2 | 2 | 1.585 |
| laugh     | 3 | 3 | 1 |
| funny     | 2 | 4 | 1.585 |
| best      | 4 | 4 | 0.585 |
| ice cream | 3 | 3 | 1 |
| dessert   | 2 | 2 | 1.585 |
| taste     | 3 | 4 | 1 |

The IDF weights tell the model to consider `best` as less important than other terms.

## Term-frequency weights

The next piece of information to consider for your weights is how frequently a term appears within a sentence. The word `funny` appears three times in one sentence—so it would be good if you could weight `funny` so that the model knows that. You can accomplish this by creating unique weights for each sentence that combines the *term frequency* (how often a word appears within an individual document) with the IDF, like so:

$$tfidf_{t,d}=(tf_{t,d})(idf_t)$$

Now, the term `funny` in the second sentence, where it occurs three times, will be weighted more heavily than the term `funny` in the first sentence, where it only occurs once. If `best` had appeared multiple times in one sentence, it would also have a higher weight for that sentence, but the weight would be reduced by the IDF term that takes into account that `best` is a pretty common word in your collection of sentences.

The TF-IDF score will be highest for a term that occurs a lot within a small number of sentences, and lowest for a word that occurs in most or all sentences.

Now you can represent each sentence as a vector made up of the TF-IDF scores for each word:

|           | 1 | 2 | 3 | 
|-----------|---|---|---|
| Monty     | 1.585 | 0 | 0 |
| Python    | 1 | 1 | 0 | 
| sketch    | 1.585| 1.585 | 0 | 
| laugh     | 1 | 1 | 0 | 
| funny     | 0 | 4.755 | 0 | 
| best      | 0.585 | 0.585 | 0.585 | 
| ice cream | 0 | 0 | 1 | 
| dessert   | 0 | 0 | 1.585 | 
| taste     | 0 | 0 | 1 |


## Things to consider in TF-IDF

As with any feature-generation technique for text data, TF-IDF vectors are a kind of translation from human-readable language to computer-usable numeric form. Some information is inevitably lost in translation, and the usefulness of any model that you build from here on out depends on the decisions that you made during the translation step. Possible decision points include the following:

* **Which stop words to include or exclude**
* **If you should use phrases as terms:** For example, `Monty Python` instead of `Monty` and `Python`.
* **The threshold for infrequent words:** In this example, you excluded words that only occurred once. In longer documents, it may be a good idea to set a higher threshold.
* **How many terms to keep:** You kept all of the terms that fit the specified criteria (not a stop word, occurred more than once). But for bigger document collections or longer documents, this may create unfeasibly long vectors. You may want to decide to only keep the 10,000 words with the highest collection frequency scores, for example.


## Implementing TF-IDF

Now, you're all set to implement TF-IDF vectorization. As you did in the previous checkpoint for the BoW approach, you'll be using the excellent scikit-learn library for generating TF-IDF vectors of Jane Austen's *Persuasion* and Lewis Carroll's *Alice's Adventures in Wonderland*.

Before jumping into the vectorization, apply the same data-cleaning steps as in the previous checkpoint:

In [1]:
import numpy as np
import pandas as pd
import sklearn
import spacy
import re
from nltk.corpus import gutenberg
import nltk
import warnings
warnings.filterwarnings("ignore")

# nltk.download('gutenberg')
# !python -m spacy download en

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Steve\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
[x] Couldn't link model to 'en'
Creating a symlink in spacy/data failed. Make sure you have the required
permissions and try re-running the command as admin, or use a virtualenv. You
can still import the model as a module and call its load() method, or create the
symlink manually.
C:\Users\Steve\anaconda3\lib\site-packages\en_core_web_sm -->
C:\Users\Steve\anaconda3\lib\site-packages\spacy\data\en
[!] Download successful but linking failed
Creating a shortcut link for 'en' didn't work (maybe you don't have admin
permissions?), but you can still load the model via its full package name: nlp =
spacy.load('en_core_web_sm')


You do not have sufficient privilege to perform this operation.


In [2]:
# Utility function for standard text cleaning
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation that spaCy doesn't
    # recognize: the double dash --. Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = re.sub(r"(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b", " ", text)
    text = ' '.join(text.split())
    return text

In [3]:
# Load and clean the data
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [4]:
# Parse the cleaned novels. This can take some time.
nlp = spacy.load('en_core_web_sm')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [5]:
# Group into sentences
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one DataFrame
sentences = pd.DataFrame(alice_sents + persuasion_sents, columns = ["text", "author"])
sentences.head()

,text,author
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(Oh, dear, !)",Carroll


In [6]:
alice_sents

[[Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?',
  'Carroll'],
 [So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.,
  'Carroll'],
 [There was nothing so VERY remarkable in that; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself, ',
  'Carroll'],
 [Oh dear!, 'Carroll'],
 [Oh dear!, 'Carroll'],
 [I shall be late!', 'Carroll'],
 [(when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural

In [7]:
# Get rid of stop words and punctuation,
# and lemmatize the tokens
for i, sentence in enumerate(sentences["text"]):
    sentences.loc[i, "text"] = " ".join(
        [token.lemma_ for token in sentence if not token.is_punct and not token.is_stop])

Now, you can start using scikit-learn's `TfidfVectorizer` class. Note that below you'll set some parameters of the `TfidfVectorizer`. Specifically, set the following:

* `max_df=0.5`: This drops words that occur in more than half of the documents.
* `min_df=2`: This makes the vectorizer only use words that appear at least twice.
* `use_idf=True`: This makes the vectorizer use inverse document frequencies in weighting.
* `norm=u'l2'`: This applies a correction factor so that longer and shorter documents are treated equally.
* `smooth_idf=True`: This adds `1` to all document frequencies, as if an extra document existed that used every word once. This prevents divide-by-zero errors.

There are other parameters of `TfidfVectorizer` that you can set. For more information, you can look at the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). 

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_df=0.5, min_df=2, use_idf=True, norm=u'l2', smooth_idf=True)


# Applying the vectorizer
X = vectorizer.fit_transform(sentences["text"])

tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
sentences = pd.concat([tfidf_df, sentences[["text", "author"]]], axis=1)

# Keep in mind that log base 2 of 1 is 0,
# so a TF-IDF score of 0 indicates that the word was present once in that sentence.
sentences.head()

,abide,ability,able,abominate,abroad,absence,absent,absolute,absolutely,absurd,...,yer,yes,yesterday,yield,young,youth,zeal,zealous,text,author
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Alice begin tired sit sister bank have twice p...,Carroll
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,consider mind hot day feel sleepy stupid pleas...,Carroll
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,remarkable Alice think way hear Rabbit,Carroll
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,oh dear,Carroll
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,oh dear,Carroll


As you can see, you now have a dataset in an easy-to-use format: the rows have the observations and the columns have the numeric features. From now on, you can use this dataset as input to machine-learning models. So, you're jumping in the modeling phase of the data-preprocessing pipeline as shown below:

![Modeling](assets/modeling.png)

## TF-IDF in action

As you did in the previous checkpoint, you'll build some machine-learning models using your dataset. Because your features are all numerical now, you can use them directly in your models. As in the previous checkpoint, your task is to predict the author of a sentence.

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split

Y = sentences['author']
X = np.array(sentences.drop(['text','author'], 1))

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=123)

# Models
lr = LogisticRegression()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()

lr.fit(X_train, y_train)
rfc.fit(X_train, y_train)
gbc.fit(X_train, y_train)

print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', gbc.score(X_train, y_train))
print('\nTest set score:', gbc.score(X_test, y_test))

----------------------Logistic Regression Scores----------------------
Training set score: 0.9124001183782184

Test set score: 0.873058144695961
----------------------Random Forest Scores----------------------
Training set score: 0.9786919206865937

Test set score: 0.8664003550821128
----------------------Gradient Boosting Scores----------------------
Training set score: 0.8499556081680971

Test set score: 0.8331114070128717


Compared to the BoW scores of the previous checkpoint, your scores are a little bit lower this time. But at least the TF-IDF features seem to reduce the overfitting of the logistic regression.

## Example of 2-grams

You can also make use of n-grams in `TfidfVectorizer`. To do that, you need to set the `ngram_range` parameter. Below, use 2-grams as your features and apply TF-IDF vectorization. Then apply the same models above to get your predictions:

In [10]:
vectorizer = TfidfVectorizer(
    max_df=0.5, min_df=2, use_idf=True, norm=u'l2', smooth_idf=True, ngram_range=(2,2))


# Applying the vectorizer
X = vectorizer.fit_transform(sentences["text"])

tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
sentences = pd.concat([tfidf_df, sentences[["text", "author"]]], axis=1)

# Keep in mind that log base 2 of 1 is 0,
# so a TF-IDF score of 0 indicates that the word was present once in that sentence.
sentences.head()

,able bear,able persuade,absence home,absolute necessity,absolutely hopeless,accident lyme,accommodation man,account louisa,account small,account taste,...,young friend,young lady,young man,young people,young person,young sister,young woman,youth say,text,author
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Alice begin tired sit sister bank have twice p...,Carroll
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,consider mind hot day feel sleepy stupid pleas...,Carroll
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,remarkable Alice think way hear Rabbit,Carroll
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,oh dear,Carroll
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,oh dear,Carroll


Notice that you have 3,381 features now (excluding the *text* and *author* columns). As you may remember, the 2-grams features of the BoW vectorizer from the previous checkpoint consist of more than 30,000 features! This is a huge reduction in the number of features due to the values that you set for the parameters of the `TfidfVectorizer`, like `max_df` and `min_df`.

Now, run your models using the 2-grams features:

In [11]:
Y = sentences['author']
X = np.array(sentences.drop(['text','author'], 1))

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=123)

# Models
lr = LogisticRegression()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()

lr.fit(X_train, y_train)
rfc.fit(X_train, y_train)
gbc.fit(X_train, y_train)

print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', gbc.score(X_train, y_train))
print('\nTest set score:', gbc.score(X_test, y_test))

----------------------Logistic Regression Scores----------------------
Training set score: 0.8233205090263391

Test set score: 0.7780736795383932
----------------------Random Forest Scores----------------------
Training set score: 0.8828055637762652

Test set score: 0.800266311584554
----------------------Gradient Boosting Scores----------------------
Training set score: 0.7682746374667061

Test set score: 0.762982689747004


The results are slightly lower again in comparison to the BoW scores of the previous checkpoint. However, this time, the overfittings of the logistic regression and the random forest models are reduced substantially. This is basically because of the reduction in the number of features.

# Some applications of TF-IDF

So far, you've applied classification models using the TF-IDF vectors to predict the authors of the sentences. Here, you'll briefly touch upon some popular NLP applications that make use of TF-IDF vectorization. In the first one, you'll briefly review how you can measure the similarities of the documents. And in the second one, you'll explore *topic modeling*, which refers to deriving the fundamental topics of a collection of documents.

## Vector space model

By now, you've had some practice thinking of data as existing in multidimensional space. Your sentences exist in an n-dimensional space, where *n* is equal to the number of terms in your term-document matrix. The vector representation of the text is referred to as a *vector space model* (VSM). You can use this representation to compute the similarity between the sentences and a new phrase or sentence. This method is often used by search engines to match a query to possible results.

To compute the similarity of sentences to a new sentence, transform the new sentence into a vector and place it in the space. You can then calculate how different the angles are for the original vectors and the new vector, and identify the vector whose angle is closest to the new vector. Typically, this is done by calculating the cosine of the angle between the vectors. If the two vectors are identical, the angle between them will be 0°, and the cosine will be 1. If the two vectors are orthogonal, with an angle of 90°, the cosine will be 0.

If you were running a search query, then you would return sentences that are most similar to the query sentence, ordered from the highest similarity score (cosine) to the lowest. Pretty handy!

As cool as this is, there are limitations to the VSM. Because it treats each word as distinct from every other word, it can run aground on *synonyms* (treating words that mean the same thing as though they are different, like big and large). Also, because it treats all occurrences of a word as the same regardless of context, it can run aground on *polysemy*, where there are different meanings attached to the same word (for example, "I need a break" versus "I break things"). In addition, the vector space model has difficulty with very large documents because the more words a document has, the more opportunities it has to diverge from other documents in the space. This can make it difficult to see similarities.

## Latent semantic analysis

A solution to this problem is to reduce your TF-IDF-weighted term-document matrix into a lower-dimensional space. In other words, you can express the information in the matrix using fewer rows by combining the information from multiple terms into one new row or dimension. You can accomplish this using principal components analysis (PCA). *Latent semantic analysis* (LSA), also called *latent semantic indexing*, is the process of applying PCA to a TF-IDF matrix. What you get in the end is clusters of terms that presumably reflect a topic. Each document will get a score for each topic, with higher scores indicating that the document is relevant to the topic. Documents can pertain to more than one topic.

LSA is handy when your corpus is too large to topically annotate by hand, or when you don't know what topics characterize your documents. It is also useful as a way of creating features to be used in other models.

Now, try it out! Once again, use the Gutenberg corpus. This time, focus on comparing paragraphs within *Emma*, another novel by Jane Austen.

In [14]:
# Reading in the data, in the form of paragraphs this time
nltk.download('punkt')

emma=gutenberg.paras('austen-emma.txt')

# Processing
emma_paras=[]
for paragraph in emma:
    para=paragraph[0]
    # Removing the double-dash -- from all words
    para=[re.sub(r'--','',word) for word in para]
    # Forming each paragraph into a string and adding it to the list of strings
    emma_paras.append(' '.join(para))

print(emma_paras[0:4])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Steve\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['[ Emma by Jane Austen 1816 ]', 'VOLUME I', 'CHAPTER I', 'Emma Woodhouse , handsome , clever , and rich , with a comfortable home and happy disposition , seemed to unite some of the best blessings of existence ; and had lived nearly twenty - one years in the world with very little to distress or vex her .']


In [15]:
X_train, X_test = train_test_split(emma_paras, test_size=0.4, random_state=0)

vectorizer = TfidfVectorizer(max_df=0.5, # Drop words that occur in more than half the paragraphs.
                             min_df=2, # Only use words that appear at least twice.
                             stop_words='english', 
                             lowercase=True, # Convert everything to lowercase (because Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS).
                             use_idf=True, # You definitely want to use inverse document frequencies in your weighting.
                             norm=u'l2', # Apply a correction factor so that longer paragraphs and shorter paragraphs get treated equally.
                             smooth_idf=True # Add 1 to all document frequencies, as if an extra document existed that used every word once. This prevents divide-by-zero errors.
                            )


# Applying the vectorizer
emma_paras_tfidf=vectorizer.fit_transform(emma_paras)
print("Number of features: %d" % emma_paras_tfidf.get_shape()[1])

# Splitting into training and test sets
X_train_tfidf, X_test_tfidf= train_test_split(emma_paras_tfidf, test_size=0.4, random_state=0)

# Reshape the vectorizer output into something that people can read
X_train_tfidf_csr = X_train_tfidf.tocsr()

# Number of paragraphs
n = X_train_tfidf_csr.shape[0]
# A list of dictionaries, one per paragraph
tfidf_bypara = [{} for _ in range(0,n)]
# List of features
terms = vectorizer.get_feature_names()
# For each paragraph, list the feature words and their TF-IDF scores.
for i, j in zip(*X_train_tfidf_csr.nonzero()):
    tfidf_bypara[i][terms[j]] = X_train_tfidf_csr[i, j]

# Keep in mind that log base 2 of 1 is 0, so a TF-IDF score of 0 indicates that the word was present once in that sentence.
print('Original sentence:', X_train[5])
print('Tf_idf vector:', tfidf_bypara[5])

Number of features: 1948
Original sentence: A very few minutes more , however , completed the present trial .
Tf_idf vector: {'minutes': 0.7127450310382584, 'present': 0.701423210857947}


### Dimension reduction
Okay, now you have your vectors, with one vector per paragraph. It's time to do some dimension reduction. Use the singular value decomposition (SVD) function from scikit-learn rather than PCA, because you don't want to mean-center your variables (and thus lose sparsity).

In [16]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

# The SVD data reducer. You are going to reduce the feature space from 1,379 to 130.
svd= TruncatedSVD(130)
lsa = make_pipeline(svd, Normalizer(copy=False))
# Run SVD on the training data, then project the training data.
X_train_lsa = lsa.fit_transform(X_train_tfidf)

variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100)

# Look at what sorts of paragraphs your solution considers similar for the first five identified topics.
paras_by_component=pd.DataFrame(X_train_lsa,index=X_train)
for i in range(5):
    print('-------COMPONENT {}:'.format(i))
    print(paras_by_component.loc[:,i].sort_values(ascending=False)[0:10])
    print("-------------------------------------------------------------")

Percent variance captured by all components: 45.190841129087936
-------COMPONENT 0:
" Oh !     0.999289
" Oh !     0.999289
" Oh !     0.999289
" Oh !     0.999289
" Oh !     0.999289
" Oh !"    0.999289
" Oh !     0.999289
" Oh !     0.999289
" Oh !     0.999289
" Oh !     0.999289
Name: 0, dtype: float64
-------------------------------------------------------------
-------COMPONENT 1:
" You have made her too tall , Emma ," said Mr . Knightley .                                                                                                                0.633147
" You get upon delicate subjects , Emma ," said Mrs . Weston smiling ; " remember that I am here . Mr .                                                                     0.567266
" I do not know what your opinion may be , Mrs . Weston ," said Mr . Knightley , " of this great intimacy between Emma and Harriet Smith , but I think it a bad thing ."    0.565230
" You are right , Mrs . Weston ," said Mr . Knightley warmly , " Mi

From glancing at the most representative sample paragraphs, it appears that component 0 targets the exclamation `Oh!`, component 1 seems to largely involve critical dialogue directed at or about the main character Emma, component 2 is chapter headings, component 3 is exclamations involving `Ah!`, and component 4 involves actions by or directly related to Emma. What fun! 

LSA is one of many unsupervised methods that can be applied to text data. Although it is good for dealing with synonyms, it cannot handle polysemy. For that, you'll need to try out other kinds of approaches; you'll see one of them in the next checkpoint. 

Although LSA has been presented here as an unsupervised method, it can also be used to prepare text data for classification in supervised learning. In that case, the goal would be to use LSA to arrive at a smaller set of features that can be used to build a supervised model that classifies text into prelabeled categories.

## 1) Converting words or sentences into numeric vectors is fundamental when working with text data. To make sure that you have a solid handle on how these vectors work, generate the TF-IDF vectors for the last three sentences of the example from the beginning of this checkpoint (from the BoW revisited: TF-IDF section). If you are feeling uncertain, have your mentor walk you through it.

In [1]:
import numpy as np
import pandas as pd
import sklearn
import spacy
import re
from nltk.corpus import gutenberg
import nltk
import warnings
warnings.filterwarnings("ignore")

In [2]:
# s1 = "The best Monty Python sketch is the one about the dead parrot; I laughed so hard."
# s2 = "I laugh when I think about Python's Ministry of Silly Walks sketch; it is funny, funny, funny, the best!"
# s3 = "Chocolate is the best ice cream dessert topping, with a great taste."
s4 = "The Lumberjack Song is the funniest Monty Python bit; I can't think of it without laughing."
s5 = "I would rather put strawberries on my ice cream for dessert; they have the best taste."
s6 = "The taste of caramel is a fantastic accompaniment to tasty mint ice cream."

In [3]:
# Parse the cleaned novels. This can take some time.
nlp = spacy.load('en_core_web_sm')
com_doc = nlp(s4)
ice_doc = nlp(s5 + s6)

In [4]:
# Group into sentences
com_sents = [[sent, "comedy"] for sent in com_doc.sents]
ice_sents = [[sent, "ice cream"] for sent in ice_doc.sents]

# Combine the sentences from the two novels into one DataFrame
sentences = pd.DataFrame(com_sents + ice_sents, columns = ["text", "type"])
sentences.head()

,text,type
0,"(The, Lumberjack, Song, is, the, funniest, Mon...",comedy
1,"(I, would, rather, put, strawberries, on, my, ...",ice cream
2,"(The, taste, of, caramel, is, a, fantastic, ac...",ice cream


In [5]:
# Get rid of stop words and punctuation,
# and lemmatize the tokens
for i, sentence in enumerate(sentences["text"]):
    sentences.loc[i, "text"] = " ".join(
        [token.lemma_ for token in sentence if not token.is_punct and not token.is_stop])

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=True, norm=u'l2', smooth_idf=True)

# Applying the vectorizer
X = vectorizer.fit_transform(sentences["text"])

tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
sentences = pd.concat([tfidf_df, sentences[["text", "type"]]], axis=1)

# Keep in mind that log base 2 of 1 is 0,
# so a TF-IDF score of 0 indicates that the word was present once in that sentence.
sentences.head()

,accompaniment,bit,caramel,cream,dessert,fantastic,funny,good,ice,laugh,...,mint,monty,python,song,strawberry,taste,tasty,think,text,type
0,0.000000,0.353553,0.000000,0.000000,0.000000,0.000000,0.353553,0.000000,0.000000,0.353553,...,0.000000,0.353553,0.353553,0.353553,0.000000,0.000000,0.000000,0.353553,Lumberjack Song funny Monty Python bit think l...,comedy
1,0.000000,0.000000,0.000000,0.349498,0.459548,0.000000,0.000000,0.459548,0.349498,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.459548,0.349498,0.000000,0.000000,strawberry ice cream dessert good taste,ice cream
2,0.385323,0.000000,0.385323,0.293048,0.000000,0.385323,0.000000,0.000000,0.293048,0.000000,...,0.385323,0.000000,0.000000,0.000000,0.000000,0.293048,0.385323,0.000000,taste caramel fantastic accompaniment tasty mi...,ice cream


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split


In [9]:
Y = sentences['type']
X = np.array(sentences.drop(['text','type'], 1))

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=66)

# Models
lr = LogisticRegression()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()

lr.fit(X_train, y_train)
rfc.fit(X_train, y_train)
gbc.fit(X_train, y_train)

print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', gbc.score(X_train, y_train))
print('\nTest set score:', gbc.score(X_test, y_test))

----------------------Logistic Regression Scores----------------------
Training set score: 1.0

Test set score: 1.0
----------------------Random Forest Scores----------------------
Training set score: 1.0

Test set score: 1.0
----------------------Gradient Boosting Scores----------------------
Training set score: 1.0

Test set score: 1.0


## 2) In the 2-grams example above, you only used 2-grams as your features. This time, use both 1-grams and 2-grams together as your feature set. Run the same models as in the example and compare the results.

In [10]:
vectorizer = TfidfVectorizer(use_idf=True, norm=u'l2', smooth_idf=True, ngram_range=(2,2))

# Applying the vectorizer
X = vectorizer.fit_transform(sentences["text"])

tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
sentences = pd.concat([tfidf_df, sentences[["text", "type"]]], axis=1)

# Keep in mind that log base 2 of 1 is 0,
# so a TF-IDF score of 0 indicates that the word was present once in that sentence.
sentences.head()

,accompaniment tasty,bit think,caramel fantastic,cream dessert,dessert good,fantastic accompaniment,funny monty,good taste,ice cream,lumberjack song,mint ice,monty python,python bit,song funny,strawberry ice,taste caramel,tasty mint,think laugh,text,type
0,0.000000,0.377964,0.000000,0.000000,0.000000,0.000000,0.377964,0.000000,0.000000,0.377964,0.000000,0.377964,0.377964,0.377964,0.000000,0.000000,0.000000,0.377964,Lumberjack Song funny Monty Python bit think l...,comedy
1,0.000000,0.000000,0.000000,0.467351,0.467351,0.000000,0.000000,0.467351,0.355432,0.000000,0.000000,0.000000,0.000000,0.000000,0.467351,0.000000,0.000000,0.000000,strawberry ice cream dessert good taste,ice cream
2,0.389888,0.000000,0.389888,0.000000,0.000000,0.389888,0.000000,0.000000,0.296520,0.000000,0.389888,0.000000,0.000000,0.000000,0.000000,0.389888,0.389888,0.000000,taste caramel fantastic accompaniment tasty mi...,ice cream


In [11]:
Y = sentences['type']
X = np.array(sentences.drop(['text','type'], 1))

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=66)

# Models
lr = LogisticRegression()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()

lr.fit(X_train, y_train)
rfc.fit(X_train, y_train)
gbc.fit(X_train, y_train)

print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', gbc.score(X_train, y_train))
print('\nTest set score:', gbc.score(X_test, y_test))

----------------------Logistic Regression Scores----------------------
Training set score: 1.0

Test set score: 1.0
----------------------Random Forest Scores----------------------
Training set score: 1.0

Test set score: 0.0
----------------------Gradient Boosting Scores----------------------
Training set score: 1.0

Test set score: 1.0


In [12]:
vectorizer = TfidfVectorizer(use_idf=True, norm=u'l2', smooth_idf=True, ngram_range=(1,2))

# Applying the vectorizer
X = vectorizer.fit_transform(sentences["text"])

tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
sentences = pd.concat([tfidf_df, sentences[["text", "type"]]], axis=1)

# Keep in mind that log base 2 of 1 is 0,
# so a TF-IDF score of 0 indicates that the word was present once in that sentence.
sentences.head()

,accompaniment,accompaniment tasty,bit,bit think,caramel,caramel fantastic,cream,cream dessert,dessert,dessert good,...,strawberry,strawberry ice,taste,taste caramel,tasty,tasty mint,think,think laugh,text,type
0,0.000000,0.000000,0.258199,0.258199,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.258199,0.258199,Lumberjack Song funny Monty Python bit think l...,comedy
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.249204,0.327673,0.327673,0.327673,...,0.327673,0.327673,0.249204,0.000000,0.000000,0.000000,0.000000,0.000000,strawberry ice cream dessert good taste,ice cream
2,0.274064,0.274064,0.000000,0.000000,0.274064,0.274064,0.208433,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.208433,0.274064,0.274064,0.274064,0.000000,0.000000,taste caramel fantastic accompaniment tasty mi...,ice cream


In [13]:
Y = sentences['type']
X = np.array(sentences.drop(['text','type'], 1))

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=66)

# Models
lr = LogisticRegression()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()

lr.fit(X_train, y_train)
rfc.fit(X_train, y_train)
gbc.fit(X_train, y_train)

print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', gbc.score(X_train, y_train))
print('\nTest set score:', gbc.score(X_test, y_test))

----------------------Logistic Regression Scores----------------------
Training set score: 1.0

Test set score: 1.0
----------------------Random Forest Scores----------------------
Training set score: 1.0

Test set score: 1.0
----------------------Gradient Boosting Scores----------------------
Training set score: 1.0

Test set score: 1.0
